<a href="https://colab.research.google.com/github/alzimna/Anmul/blob/main/Chapters/Bab_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import scipy
import os
import math
import cv2
import shutil
import requests, zipfile, io
from PIL import Image
from skimage import feature, color, data
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from IPython.display import clear_output
from numpy.linalg import inv as inv
from numpy.linalg import eig as eig
from numpy import transpose as t
from numpy.linalg import det as det

#**13.3 ESTIMATION OF LOADINGS AND COMMUNALITIES**

##**13.3.1 Principal Component Method**

###Example 13.3.1

In [ ]:
url = 'https://github.com/alzimna/Anmul/raw/main/data/t13.1.xlsx'
data = pd.read_excel(url)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Kind         7 non-null      int64
 1   Intelligent  7 non-null      int64
 2   Happy        7 non-null      int64
 3   Likeable     7 non-null      int64
 4   Just         7 non-null      int64
dtypes: int64(5)
memory usage: 408.0 bytes


In [ ]:
R = data.corr()
print(np.round(R,3))

              Kind  Intelligent  Happy  Likeable   Just
Kind         1.000        0.296  0.881     0.995  0.545
Intelligent  0.296        1.000 -0.022     0.326  0.837
Happy        0.881       -0.022  1.000     0.867  0.130
Likeable     0.995        0.326  0.867     1.000  0.544
Just         0.545        0.837  0.130     0.544  1.000


In [ ]:
E = eig(R)[0]
print(np.round(sorted(E)[::-1],3))

[3.263 1.538 0.168 0.031 0.   ]


In [ ]:
ev = eig(R)[1]
c1 = ev[:,0]*(-1)
c2 = ev[:,1]
C = pd.DataFrame({
    "c1" : c1,
    "c2" : c2

 })
C
D = np.diag(np.sqrt(E[:2]))
D

Lambda_PC = C @ D
Lambda_PC = Lambda_PC.rename(columns={
    0 : "c1",
    1 : "c2"
})

In [ ]:
communal = Lambda_PC.iloc[:,0]**2+Lambda_PC.iloc[:,1]**2
communal

Psi = R.values-Lambda_PC @ t(Lambda_PC)
Psi_i = np.diag(Psi)

In [ ]:
df = pd.DataFrame({
  "Communalities" : np.round(communal,3),
  "Specific Variances" : np.round(Psi_i,3)
})
df = pd.concat([np.round(Lambda_PC,3),df],axis = 1)
df

,c1,c2,Communalities,Specific Variances
0,0.969,-0.231,0.993,0.007
1,0.519,0.807,0.921,0.079
2,0.785,-0.587,0.960,0.040
3,0.971,-0.210,0.987,0.013
4,0.704,0.667,0.940,0.060


## **13.3.2 Principal Factor Method**

### Example 13.3.2

In [ ]:
url = 'https://github.com/alzimna/Anmul/raw/main/data/t13.1.xlsx'
data = pd.read_excel(url)

R = data.corr()
R

,Kind,Intelligent,Happy,Likeable,Just
Kind,1.000000,0.295536,0.880572,0.995429,0.544567
Intelligent,0.295536,1.000000,-0.021744,0.326164,0.837288
Happy,0.880572,-0.021744,1.000000,0.866667,0.130337
Likeable,0.995429,0.326164,0.866667,1.000000,0.544016
Just,0.544567,0.837288,0.130337,0.544016,1.000000


In [ ]:
hi_square_PF = []
for i in range(5) :
  cek = R.iloc[i,:]
  cek = sorted(cek)
  hi_square_PF.append(cek[-2])
hi_square_PF = pd.DataFrame({
    "hi_square" : hi_square_PF
})
hi_square_PF

,hi_square
0,0.995429
1,0.837288
2,0.880572
3,0.995429
4,0.837288


In [ ]:
I = np.diag([1]*data.shape[1])
R_Psi = R.values-I+np.diag(hi_square_PF)
E = eig(R_Psi)[0]
print(np.round(sorted(E)[::-1],3))

[ 7.17   0.573 -0.808 -0.958 -1.   ]


In [ ]:
ev = eig(R_Psi)[1]
c1 = ev[:,0]
c2 = ev[:,1]*(-1)
C = pd.DataFrame({
    "c1" : c1,
    "c2" : c2

 })
C

D = np.diag(np.sqrt(E[:2]))
D

Lambda_PF = C @ D
Lambda_PF = Lambda_PF.rename(columns={
    0 : "f1",
    1 : "f2"
})

In [ ]:
tabel = pd.concat([np.round(Lambda_PC,3),np.round(Lambda_PF,3),np.round(hi_square_PF,3)],axis = 1)
tabel

,c1,c2,f1,f2,hi_square
0,0.969,-0.231,-1.283,0.195,0.995
1,0.519,0.807,-1.074,-0.469,0.837
2,0.785,-0.587,-1.160,0.385,0.881
3,0.971,-0.210,-1.285,0.182,0.995
4,0.704,0.667,-1.171,-0.365,0.837


## **13.3.3 Iterated Principal Factor Method**

In [ ]:
url = 'https://github.com/alzimna/Anmul/raw/main/data/t7.1.xlsx'
data = pd.read_excel(url)

R = data.corr()
R

,y1,y2,X1,X2,X3,X4,X5,X6,X7,X8
y1,1.000000,0.558472,0.216284,0.104148,0.200553,-0.044389,0.129532,0.032747,-0.070235,0.092220
y2,0.558472,1.000000,-0.086262,0.130921,0.199264,-0.165783,0.171805,0.239287,0.156460,0.059918
X1,0.216284,-0.086262,1.000000,0.158101,0.703601,-0.290648,-0.452928,-0.344532,-0.114286,0.682477
X2,0.104148,0.130921,0.158101,1.000000,0.486469,-0.034647,-0.164630,0.004830,0.418528,0.369347
X3,0.200553,0.199264,0.703601,0.486469,1.000000,-0.310260,-0.336894,-0.190395,0.299918,0.874888
X4,-0.044389,-0.165783,-0.290648,-0.034647,-0.310260,1.000000,-0.428126,-0.587294,-0.091106,-0.262774
X5,0.129532,0.171805,-0.452928,-0.164630,-0.336894,-0.428126,1.000000,0.823014,0.234037,-0.297725
X6,0.032747,0.239287,-0.344532,0.004830,-0.190395,-0.587294,0.823014,1.000000,0.445297,-0.173440
X7,-0.070235,0.156460,-0.114286,0.418528,0.299918,-0.091106,0.234037,0.445297,1.000000,0.289186
X8,0.092220,0.059918,0.682477,0.369347,0.874888,-0.262774,-0.297725,-0.173440,0.289186,1.000000


In [ ]:
hi_square_PF = []
R_inv = inv(R)
for i in range(10) :
  hi_square_PF.append(1-1/R_inv[i,i])
initial_commun = hi_square_PF

R_Psi_1 = R.values-np.diag(np.diag(R))+np.diag(hi_square_PF)

E = eig(R_Psi_1)[0]

ev = eig(R_Psi_1)[1]
C = ev[:,:4]
D = np.diag(np.sqrt(E[:4]))
Lambda_PF = C @ D

hi_square_PF = []
for i in range(10) :
    temp = 0
    for j in range(4) :
      temp+=Lambda_PF[i,j]**2
    hi_square_PF.append(temp)
hi_square_PF

[0.6072827777564477,
 0.5533686247070694,
 0.7959587743145244,
 0.38587920438765544,
 0.9259481420265129,
 0.7809806170892124,
 0.7826640512090802,
 0.9156353501221425,
 0.6875873961179921,
 0.7976289354513495]

In [ ]:
R_Psi = dict()
R_Psi[1] = R_Psi_1
k = 100
for i in range(2,k) :
  R_Psi[i] = R_Psi[i-1]-np.diag(np.diag(R_Psi[i-1]))+np.diag(hi_square_PF)
  E = eig(R_Psi[i])[0]

  ev = eig(R_Psi[i])[1]
  C = ev[:,:4]
  D = np.diag(np.sqrt(E[:4]))
  Lambda_PF = C @ D

  hi_square_PF = []
  for _ in range(10) :
      temp = 0
      for j in range(4) :
        temp+=Lambda_PF[_,j]**2
      hi_square_PF.append(temp)
  if max(hi_square_PF)>1 :
    break
  # print(hi_square_PF)
print("hi akan konvergen ketika i = ",i)

hi akan konvergen ketika i =  48


In [ ]:
final_commun = hi_square_PF
f1 = Lambda_PF[:,0]
f2 = Lambda_PF[:,1]
f3 = Lambda_PF[:,2]*(-1)
f4 = Lambda_PF[:,3]*(-1)

tabel=pd.DataFrame({
    "f1" : np.round(f1,2),
    "f2" : np.round(f2,2),
    "f3" : np.round(f3,2),
    "f4" : np.round(f4,2),
    "Initial Communalities" : np.round(initial_commun,2),
    "Final Communalities" : np.round(final_commun,2),
})
tabel

,f1,f2,f3,f4,Initial Communalities,Final Communalities
0,0.22,0.30,0.92,0.12,0.57,1.00
1,0.07,0.40,0.43,0.20,0.54,0.39
2,0.80,-0.04,0.05,-0.38,0.78,0.79
3,0.41,0.22,-0.11,0.37,0.40,0.36
4,0.94,0.28,-0.07,0.05,0.88,0.98
5,-0.13,-0.67,0.10,0.56,0.77,0.79
6,-0.55,0.66,0.03,-0.11,0.79,0.75
7,-0.45,0.88,-0.14,-0.08,0.87,0.99
8,0.13,0.54,-0.37,0.54,0.66,0.74
9,0.84,0.21,-0.17,-0.02,0.80,0.78


##  **13.3.4 Maximum Likelihood Method**

### Example 13.3.4

In [ ]:
!pip install factor_analyzer
from factor_analyzer import FactorAnalyzer
fa = FactorAnalyzer(n_factors=4,method = 'ml',rotation = None)
fa.fit(data)
f = np.round(fa.loadings_,2)
commun = np.round(fa.get_communalities(),2)

In [ ]:
f = pd.DataFrame(f)
f = f.rename(columns = {0 : "f1",1 : "f2",2 : "f3",3 : "f4"})

commun = pd.DataFrame(commun)
commun = commun.rename(columns = {0 : "Communalties"})
tabel = pd.concat([f,commun],axis = 1)
tabel

,f1,f2,f3,f4,Communalties
0,0.01,-0.05,1.00,0.00,1.00
1,0.19,0.14,0.56,0.13,0.39
2,-0.51,0.52,0.25,-0.35,0.71
3,-0.14,0.44,0.13,0.37,0.36
4,-0.45,0.84,0.24,0.04,0.97
5,-0.46,-0.62,-0.07,0.54,0.89
6,0.86,0.02,0.12,-0.03,0.75
7,0.95,0.27,0.04,0.03,0.98
8,0.30,0.50,-0.05,0.65,0.77
9,-0.41,0.78,0.13,0.04,0.79
